In [1]:
from sqlalchemy import create_engine
import dbconfig
import pandas as pd

In [2]:
engine = create_engine('postgresql://{conf.user}:{conf.password}@{conf.host}:5432/{conf.database}'.format(conf=dbconfig))

In [3]:
pop_housing_sql = """SELECT census.*, groups.area from shape_files.census_pop_housing as census
                    JOIN shape_files.census_blocks_groups as groups
                    on census.tract = groups.tract
                    and census.block_group = groups.blkgrp;"""

In [4]:
pop_housing_raw = pd.read_sql_query(pop_housing_sql, con=engine)

# Raw census data

In [5]:
pop_housing_raw.head()

,index,tract,block_group,H0030001,H0030002,H0030003,H0040002,H0040004,H0050002,H0060002,...,P0120045,P0120046,P0120047,P0120048,P0120049,P0180001,P0180002,P0180003,P0180004,area
0,0,000200,1,576,516,60,1,513,50,9,...,13,12,17,13,7,516,255,34,221,3123974.646
1,1,000700,1,142,94,48,0,94,16,77,...,1,2,2,1,0,94,29,25,4,1076454.735
2,2,000700,2,1182,974,208,28,944,119,726,...,7,7,1,14,14,974,114,93,21,3162576.446
3,3,000700,3,607,512,95,39,468,27,307,...,7,7,2,2,8,512,42,33,9,4203521.971
4,4,000900,1,684,148,536,11,136,43,42,...,6,2,6,0,1,148,58,17,41,1459837.484


# Calculating census features

list of feature description and calculation can be found in folder docs/data_dictionaries

features are claculated for each pair of census tract and block 

In [5]:
features = pd.DataFrame({   'tract' : pop_housing_raw.tract,
                            'block_group' : pop_housing_raw.block_group,
                            'housing_density': pop_housing_raw.H0030001/pop_housing_raw.area,
                            'rate_occupied_units': pop_housing_raw.H0030002/pop_housing_raw.H0030001,
                            'rate_vacant_units': pop_housing_raw.H0030003/pop_housing_raw.H0030001,
                            'rate_mortgage_or_loan' : pop_housing_raw.H0040002/pop_housing_raw.H0030001,
                            'rate_renter_occupied' : pop_housing_raw.H0040004/pop_housing_raw.H0030001,
                            'rate_for_rent' : pop_housing_raw.H0050002/pop_housing_raw.H0030001,
                            'rate_white_householder' : pop_housing_raw.H0060002/pop_housing_raw.P0180001,
                            'rate_black_householder' : pop_housing_raw.H0060003/pop_housing_raw.P0180001,
                            'rate_native_householder' : (pop_housing_raw.H0060004+pop_housing_raw.H0060006)/pop_housing_raw.P0180001,
                            'rate_asian_householder' : pop_housing_raw.H0060005/pop_housing_raw.P0180001,
                            'rate_other_race_householder' : pop_housing_raw.H0060007/pop_housing_raw.P0180001,
                            'rate_pop_occupied_units' : pop_housing_raw.H0100001/pop_housing_raw.P0010001,
                            'rate_1_per_household' : pop_housing_raw.H0130002/pop_housing_raw.P0180001,
                            'rate_2_per_household' : pop_housing_raw.H0130003/pop_housing_raw.P0180001,
                            'rate_3_per_household' : pop_housing_raw.H0130004/pop_housing_raw.P0180001,
                            'rate_4_per_household' : pop_housing_raw.H0130005/pop_housing_raw.P0180001,
                            'rate_5_per_household' : pop_housing_raw.H0130006/pop_housing_raw.P0180001,
                            'rate_6_per_household' : pop_housing_raw.H0130007/pop_housing_raw.P0180001,
                            'rate_7_plus_per_household' : pop_housing_raw.H0130008/pop_housing_raw.P0180001,
                            'rate_owner_occupied' : pop_housing_raw.H0140002/pop_housing_raw.H0030001,
                            'rate_owner_occupied_white' : pop_housing_raw.H0140003/pop_housing_raw.H0140002,
                            'rate_owner_occupied_black' : pop_housing_raw.H0140004/pop_housing_raw.H0140002,
                            'rate_owner_occupied_native' : (pop_housing_raw.H0140005+pop_housing_raw.H0140007)/pop_housing_raw.H0140002,
                            'rate_owner_occupied_asian' : pop_housing_raw.H0140006/pop_housing_raw.H0140002,
                            'rate_owner_occupied_other_race' : pop_housing_raw.H0140008/pop_housing_raw.H0140002,
                            'rate_renter_occupied_white' : pop_housing_raw.H0140011/pop_housing_raw.H0040004,
                            'rate_renter_occupied_black' : pop_housing_raw.H0140012/pop_housing_raw.H0040004,
                            'rate_renter_occupied_native' : (pop_housing_raw.H0140013+pop_housing_raw.H0140015)/pop_housing_raw.H0040004,
                            'rate_renter_occupied_asian' : pop_housing_raw.H0140014/pop_housing_raw.H0040004,
                            'rate_renter_occupied_other' : pop_housing_raw.H0140016/pop_housing_raw.H0040004,
                            'rate_owner_occupied_hispanic' : pop_housing_raw.H0150004/pop_housing_raw.H0140002,
                            #'rate_renter_occupied_hispanic' : pop_housing_raw.H0150005/pop_housing_raw.H0040004,
                            'rate_owner_occupied_w_children' : pop_housing_raw.H0190003/pop_housing_raw.H0140002,
                            'rate_owner_occupied_no_children' : pop_housing_raw.H0190004/pop_housing_raw.H0140002,
                            'rate_renter_occupied_no_children' : 1-(pop_housing_raw.H0190006/pop_housing_raw.H0040004),
                            'rate_renter_occupied_w_children' : pop_housing_raw.H0190006/pop_housing_raw.H0040004,
                            'population_density' : pop_housing_raw.P0010001/pop_housing_raw.area,
                            'rate_white_pop' : pop_housing_raw.P0030002/pop_housing_raw.P0010001,
                            'rate_black_pop' : pop_housing_raw.P0030003/pop_housing_raw.P0010001,
                            'rate_native_pop' : (pop_housing_raw.P0030006+pop_housing_raw.P0030004)/pop_housing_raw.P0010001,
                            'rate_asian_pop' : pop_housing_raw.P0030005/pop_housing_raw.P0010001,
                            'rate_other_race_pop' : pop_housing_raw.P0030007/pop_housing_raw.P0010001,
                            'rate_pop_over_18' : pop_housing_raw.P0110001/pop_housing_raw.P0010001,
                            'rate_male_under_18' : (pop_housing_raw.P0120003+pop_housing_raw.P0120004+pop_housing_raw.P0120005+pop_housing_raw.P0120006)/pop_housing_raw.P0010001,                         
                            'rate_male_18_35' : pop_housing_raw[['P0120007','P0120008','P0120009','P0120010','P0120011','P0120012']].sum(axis=1)/pop_housing_raw.P0010001,                         
                            'rate_male_35_50' : pop_housing_raw[['P0120013','P0120014','P0120015']].sum(axis=1)/pop_housing_raw.P0010001,
                            'rate_male_50_75' : pop_housing_raw[['P0120016',	'P0120017',	'P0120018',	'P0120019',	'P0120020',	'P0120021',	'P0120022']].sum(axis=1)/pop_housing_raw.P0010001,
                            'rate_male_over_75' : pop_housing_raw[['P0120023','P0120024','P0120025']].sum(axis=1)/pop_housing_raw.P0010001,                         
                            'rate_female_under_18' : pop_housing_raw[['P0120027','P0120028','P0120029','P0120030']].sum(axis=1)/pop_housing_raw.P0010001,  
                            'rate_female_18_35' : pop_housing_raw[['P0120031',	'P0120032',	'P0120033',	'P0120034',	'P0120035',	'P0120036']].sum(axis=1)/pop_housing_raw.P0010001,                         
                            'rate_female_35_50' : pop_housing_raw[['P0120037',	'P0120038',	'P0120039']].sum(axis=1)/pop_housing_raw.P0010001,
                            'rate_female_50_75' : pop_housing_raw[['P0120040',	'P0120041',	'P0120042',	'P0120043',	'P0120044',	'P0120045',	'P0120046']].sum(axis=1)/pop_housing_raw.P0010001,
                            'rate_male_over_75' : pop_housing_raw[['P0120047','P0120048','P0120049']].sum(axis=1)/pop_housing_raw.P0010001,
                            'rate_households' : pop_housing_raw.P0180001/pop_housing_raw.H0030001})

In [7]:
features

,block_group,housing_density,population_density,rate_1_per_household,rate_2_per_household,rate_3_per_household,rate_4_per_household,rate_5_per_household,rate_6_per_household,rate_7_plus_per_household,...,rate_renter_occupied_black,rate_renter_occupied_native,rate_renter_occupied_no_children,rate_renter_occupied_other,rate_renter_occupied_w_children,rate_renter_occupied_white,rate_vacant_units,rate_white_householder,rate_white_pop,tract
0,1,0.000184,0.000342,0.445736,0.265504,0.176357,0.065891,0.019380,0.019380,0.007752,...,0.951267,0.001949,0.711501,0.001949,0.288499,0.017544,0.104167,0.017442,0.022472,000200
1,1,0.000132,0.001477,0.563830,0.329787,0.063830,0.031915,0.010638,0.000000,0.000000,...,0.117021,0.000000,0.936170,0.010638,0.063830,0.819149,0.338028,0.819149,0.380503,000700
2,2,0.000374,0.000398,0.757700,0.211499,0.021561,0.004107,0.002053,0.001027,0.002053,...,0.181144,0.000000,0.979873,0.009534,0.020127,0.741525,0.175973,0.745380,0.746032,000700
3,3,0.000144,0.000154,0.859375,0.121094,0.011719,0.001953,0.005859,0.000000,0.000000,...,0.371795,0.004274,0.985043,0.010684,0.014957,0.574786,0.156507,0.599609,0.584877,000700
4,1,0.000469,0.000290,0.486486,0.310811,0.135135,0.047297,0.013514,0.006757,0.000000,...,0.720588,0.007353,0.764706,0.000000,0.235294,0.264706,0.783626,0.283784,0.285377,000900
5,2,0.000268,0.000688,0.611276,0.240356,0.074184,0.047478,0.011869,0.002967,0.011869,...,0.672727,0.003636,0.778182,0.000000,0.221818,0.301818,0.296451,0.418398,0.340391,000900
6,2,0.000003,0.000008,0.164384,0.497717,0.146119,0.118721,0.050228,0.018265,0.004566,...,0.000000,0.000000,0.666667,0.000000,0.333333,1.000000,0.102459,1.000000,0.988889,020501
7,1,0.000003,0.000008,0.185606,0.429924,0.168561,0.142045,0.045455,0.018939,0.009470,...,0.000000,0.035714,0.857143,0.000000,0.142857,0.964286,0.045208,0.992424,0.985778,020502
8,1,0.000488,0.000607,0.475771,0.207048,0.176211,0.070485,0.039648,0.017621,0.013216,...,0.843318,0.000000,0.668203,0.004608,0.331797,0.115207,0.428212,0.132159,0.101215,001000
9,2,0.000386,0.000509,0.614964,0.293796,0.049270,0.029197,0.007299,0.000000,0.005474,...,0.344828,0.000000,0.868966,0.004598,0.131034,0.597701,0.234637,0.653285,0.559915,001000


In [10]:
features.to_sql('census_features', engine, schema='shape_files', if_exists='replace', index=False)